In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark: SparkSession = SparkSession.builder.master('local[*]').getOrCreate()

# Import and Processing of data

In [ ]:
data = spark.read.csv('./data/imdb-reviews-pt-br.csv', header=True, escape='\"', inferSchema=True)

In [ ]:
print(f'Número de linhas: {data.count()} | Número de colunas: {len(data.columns)}')

In [ ]:
data.printSchema()

In [ ]:
data.show()

In [ ]:
data.filter(data.id == 190).select('text_pt').show(truncate=False)

In [ ]:
data.filter(data.id == 12427).select('text_pt').show(truncate=False)

In [ ]:
data.groupBy('sentiment').count().show()

# WordCloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
sample = data.select('text_pt').sample(fraction=0.1, seed=101)

In [ ]:
all = [text['text_pt'] for text in sample.collect()]

In [ ]:
wordcloud = WordCloud(
  background_color='white',
  width=1920,
  height=1080,
  collocations=False,
  prefer_horizontal=1
).generate(str(all))

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Limpeza

In [ ]:
import string

In [ ]:
string.punctuation

In [ ]:
sample = spark.createDataFrame(
  [
    ("Oi, JP! Blz?",),
    ("$$$\\ |~ Parabéns ~| \\$$$",),
    ("(#amovc #paz&amor ^.^)",),
    ("\"bora *_* \"",),
    ("=>->'...``` vc foi selecionad@ ´´´...'<=<-",),
    ("{comprar: arroz; feijão e pepino} //",),
    ("!\"#$&'()*+,-./:;<=>?@[\]^_`{|}~",),
    ("ana@gmail.com",)
  ],
  ["texts"]
)

In [ ]:
import pyspark.sql.functions as f

In [ ]:
sample = sample.withColumn('text_regex', f.regexp_replace('texts', '[\$#,\"!%&\'()*+-./;:<=>?@^_`´{|}~\\\\]', ''))

In [ ]:
sample = sample.withColumn('clean_text', f.trim('text_regex'))

In [ ]:
sample.show(truncate=False)

In [ ]:
data = data.withColumn('text_en_regex', f.regexp_replace('text_en', '[\$#,\"!%&\'()*+-./;:<=>?@^_`´{|}~\\\\]', ''))
data = data.withColumn('text_pt_regex', f.regexp_replace('text_pt', '[\$#,\"!%&\'()*+-./;:<=>?@^_`´{|}~\\\\]', ''))

In [ ]:
data = data.withColumn('clean_text_en', f.trim('text_en_regex'))
data = data.withColumn('clean_text_pt', f.trim('text_pt_regex'))

In [ ]:
data.limit(2).show(truncate=False)

# Tokenização

In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol='clean_text_pt', outputCol='tokens')

In [ ]:
tokenized = tokenizer.transform(data)

In [ ]:
tokenized.select('clean_text_pt', 'tokens').show()

In [ ]:
from pyspark.sql.types import IntegerType

countTokens = f.udf(lambda tokens: len(tokens), IntegerType())

tokenized \
  .select('clean_text_pt', 'tokens') \
  .withColumn('freq_tokens', countTokens(f.col('tokens'))) \
  .show()

# StopWords

In [ ]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
stop_A = stopwords.words('portuguese')

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
remover = StopWordsRemover(inputCol='tokens', outputCol='final_text', stopWords=stop_A)

In [ ]:
df = remover.transform(tokenized)

In [ ]:
df.show()

In [ ]:
df \
  .select('tokens', 'final_text') \
  .withColumn('freq_tokens', countTokens(f.col('tokens'))) \
  .withColumn('freq_clean_tokens', countTokens(f.col('final_text'))) \
  .show()

# Vetorização

In [ ]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
cv = CountVectorizer(inputCol='final_text', outputCol='count_vec')

In [ ]:
model = cv.fit(df)

In [ ]:
count_vectorizer_features = model.transform(df)

In [ ]:
count_vectorizer_features.select('final_text', 'count_vec').show(truncate=False)

In [ ]:
model.vocabulary

# Hasing TF

Alternativa ao Count Vector

In [ ]:
from pyspark.ml.feature import HashingTF

In [ ]:
hashing_tf = HashingTF(inputCol='final_text', outputCol='hashing_tf')

In [ ]:
hashing_tf.setNumFeatures(50)

In [ ]:
htf_featurezed_data = hashing_tf.transform(count_vectorizer_features)

In [ ]:
htf_featurezed_data.select('final_text', 'hashing_tf').show()

# TF-IDF

Estabelecendo pesos

In [ ]:
from pyspark.ml.feature import IDF

In [ ]:
idf = IDF(inputCol='hashing_tf', outputCol='features')

In [ ]:
idf_model = idf.fit(htf_featurezed_data)

In [ ]:
tf_idf_featurized_data = idf_model.transform(htf_featurezed_data)

In [ ]:
tf_idf_featurized_data.select('final_text', 'features').show(truncate=False)

# Codificando

Variável e resposta

In [ ]:
tf_idf_featurized_data.groupBy('sentiment').count().show()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
stringindexes = StringIndexer(inputCol='sentiment', outputCol='label')

In [ ]:
data = stringindexes.fit(data).transform(data)

In [ ]:
data.show()

In [ ]:
data.groupBy('sentiment', 'label').count().show()

# Pipeline

Transformação dos dados

In [ ]:
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol = "clean_text_pt", outputCol = "tokens")
stopwords = StopWordsRemover(inputCol="tokens", outputCol="final_text")
hashingTF = HashingTF(inputCol=stopwords.getOutputCol(), outputCol="htf", numFeatures=1000)
tfidf = IDF(inputCol="htf", outputCol="features")

pipeline = Pipeline(stages = [tokenizer, stopwords, hashingTF, tfidf])

In [ ]:
transform_data = pipeline.fit(data).transform(data)

In [ ]:
transform_data.show(truncate=False)

# Model

In [ ]:
feature_df = transform_data.select('features', 'label')

In [ ]:
feature_df.printSchema()

In [ ]:
feature_df.show()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier

tokenizer = Tokenizer(inputCol = "clean_text_pt", outputCol = "tokens")
stopwords = StopWordsRemover(inputCol='tokens', outputCol='final_text', stopWords=stop_A)
hashingTF = HashingTF(inputCol=stopwords.getOutputCol(), outputCol="htf", numFeatures=1000)
tfidf = IDF(inputCol="htf", outputCol="features")
dt = DecisionTreeClassifier(featuresCol='features', labelCol='label', maxDepth=10)

pipeline = Pipeline(stages = [tokenizer, stopwords, hashingTF, tfidf, dt])

# Amostra e Treino

In [ ]:
train, test = data.randomSplit([0.7, 0.3], seed=101)

In [ ]:
dt_model = pipeline.fit(train)

In [ ]:
predictions = dt_model.transform(test)

In [ ]:
predictions.select('label', 'prediction').show()

# Acuracia

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
print(f'Acuracia = {accuracy}')

In [ ]:
new_base = spark.createDataFrame([
    (1, "Esse é sem dúvida o pior filme que já vi, odiei a atuação dos atores."),
    (0, "Eu amei esse filme, excelente atuação!"),
  ], ["id", "clean_text_pt"])

In [ ]:
new_class = dt_model.transform(new_base)

In [ ]:
new_class.show()